In [156]:
import pandas as pd
import numpy as np
import pickle

In [157]:
World_happiness= pd.read_csv('cleaned_version')

1. get dummy variables of categorical data using onehotencoder

In [158]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
Arrays= enc.fit_transform(World_happiness[["Country_Name","Regional_Indicator"]]).toarray()

In [159]:
enc = OneHotEncoder(handle_unknown='ignore')
Arrays= enc.fit_transform(World_happiness[["Country_Name","Regional_Indicator"]]).toarray()

In [160]:
columns=enc.categories_
Column_Arrays= np.concatenate(np.array(columns,dtype=object))
#creating one array out of the column name

In [161]:
Dummie=pd.DataFrame(Arrays,columns=Column_Arrays)

In [162]:
World_happiness1= pd.concat([World_happiness ,Dummie], axis=1)

In [163]:
World_happiness1.drop(["Country_Name", "Regional_Indicator"], axis=1, inplace=True)

3. Seperating the data

In [164]:
X1= World_happiness1.drop("Life_Ladder", axis=1)
y1= World_happiness1['Life_Ladder'].values

2. Since age column is not in the same range as the other data, we need to standarize our data

3. Doing train test split 

In [178]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [166]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X= scaler.fit_transform(X_train)
y= scaler.fit_transform(y_train.reshape(-1, 1))


# Agenda:
1. looking for multicoliniraiety 
2. Linear regression
3. lasso regression
4. ridge regression
5. random forest
6. adaboost regressor
7. SVM regressor 

### 1. Variance Inflation Factor(multicollinearity detection)

In [167]:
X2= World_happiness[["Year","Log_GDP_Per_Capita","Social_Support","Healthy_Life_Expectancy_At_Birth","Freedom_To_Make_Life_Choices","Generosity","Perceptions_Of_Corruption","Negative_Affect","Confidence_In_National_Government","Life_Ladder"]]

In [168]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["feature"] = X2.columns
vif_data["VIF"] = [variance_inflation_factor(X2.values, i) for i in range(len(X2.columns))]

In [169]:
vif_data
#From the table down below we can conclude we have severe multicollineairty (5 < VIF) in all but one feature (generosity) 

,feature,VIF
0,Year,283.368456
1,Log_GDP_Per_Capita,284.497183
2,Social_Support,123.121342
3,Healthy_Life_Expectancy_At_Birth,278.598738
4,Freedom_To_Make_Life_Choices,58.136353
5,Generosity,1.211377
6,Perceptions_Of_Corruption,31.734353
7,Negative_Affect,14.972404
8,Confidence_In_National_Government,14.764294
9,Life_Ladder,99.067344


### 2. Linear regression
a: Statmodels

In [170]:
import statsmodels.api as sm
X_sm= X1 = sm.add_constant(X1)
model = sm.OLS(y1,X_sm)

### Please remove double hashtag below and rerun the cell to see full report, it has been commneted out due to big length

In [171]:
##model.fit().summary()

1. Ajusted R - 0.896 (which means the independent variables can explain almost 90 percent of variability in dependent the variable(life ladder))
2. we cannot take our coefficient values seriously due to hufe multicollinearity, hence, even p-values are neglected 
3. the model has a Kurtosis:	4.258, which means the distribustion tails is fatter than a normal ditribustion 	

b: Scikit-learn

In [186]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [187]:
reg.score(X_train,y_train)

0.9080845330044763

In [183]:
reg = LinearRegression()
reg.fit(X_train,y_train)
np.mean(cross_val_score(reg, X_train.values, y_train,scoring= 'neg_mean_absolute_error', cv=3)).round(5)
# we are off by 0.29594 life ladder points in our train data

-0.29594

2. Lasso regression

In [175]:
from sklearn import linear_model
clf = linear_model.Lasso()
np.mean(cross_val_score(clf, X, y, scoring= 'r2', cv=4))



-0.0063632567129319195

In [176]:
clf.fit(X,y)
clf.score(X,y)


0.0

In [177]:
World_happiness

,Country_Name,Regional_Indicator,Year,Life_Ladder,Log_GDP_Per_Capita,Social_Support,Healthy_Life_Expectancy_At_Birth,Freedom_To_Make_Life_Choices,Generosity,Perceptions_Of_Corruption,Positive_Affect,Negative_Affect,Confidence_In_National_Government
0,Afghanistan,South Asia,2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072
1,Afghanistan,South Asia,2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545
2,Afghanistan,South Asia,2010,4.758381,7.613900,0.539075,51.099998,0.600127,0.121316,0.706766,0.516907,0.275324,0.299357
3,Afghanistan,South Asia,2011,3.831719,7.581259,0.521104,51.400002,0.495901,0.163571,0.731109,0.479835,0.267175,0.307386
4,Afghanistan,South Asia,2012,3.782938,7.660506,0.520637,51.700001,0.530935,0.237588,0.775620,0.613513,0.267919,0.435440
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,Zimbabwe,Sub-Saharan Africa,2018,3.616480,7.783066,0.775388,52.625000,0.762675,-0.051219,0.844209,0.657524,0.211726,0.550508
2195,Zimbabwe,Sub-Saharan Africa,2019,2.693523,7.697755,0.759162,53.099998,0.631908,-0.047464,0.830652,0.658434,0.235354,0.456455
2196,Zimbabwe,Sub-Saharan Africa,2020,3.159802,7.596050,0.717243,53.575001,0.643303,0.006313,0.788523,0.660658,0.345736,0.577302
2197,Zimbabwe,Sub-Saharan Africa,2021,3.154578,7.656878,0.685151,54.049999,0.667636,-0.075575,0.756945,0.609917,0.241682,0.665703
